In [1]:


import csv
from io import StringIO
import requests
import pandas as pd
import config


In [5]:

url = 'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&outputsize=full&symbol=IBM&datatype=csv&apikey=' + config.api_key

r = requests.get(url)

csv_data = r.text

df = pd.read_csv(StringIO(csv_data))

df

,timestamp,open,high,low,close,volume
0,2024-03-26,189.15,189.985,188.53,188.67,4228993
1,2024-03-25,190.26,190.820,188.75,188.79,3718289
2,2024-03-22,192.00,192.985,190.51,190.84,3988398
3,2024-03-21,193.00,193.370,190.01,191.90,6013561
4,2024-03-20,192.87,193.980,191.31,193.96,3238643
...,...,...,...,...,...,...
6134,1999-11-05,92.75,92.940,90.19,90.25,13737600
6135,1999-11-04,94.44,94.440,90.00,91.56,16697600
6136,1999-11-03,95.87,95.940,93.50,94.37,10369100
6137,1999-11-02,96.75,96.810,93.69,94.81,11105400
